# Analysing Catalogs

In [2]:
emu_catalog_path = "/home/polaris/Laboratório Astrofísica/data/G23-ASKAP-EMUES-master-cat.fits"
gama_catalog_path = "/home/polaris/Laboratório Astrofísica/data/gkvInputCatv02.fits"

In [3]:
from astropy.io import fits

emu_catalog = fits.open(emu_catalog_path)
gama_catalog = fits.open(gama_catalog_path)

emu_catalog_data = emu_catalog[1].data
gama_catalog_data = gama_catalog[1].data

emu_catalog_header = emu_catalog[1].header
gama_catalog_header = gama_catalog[1].header

emu_catalog_header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / 8-bit bytes                                    
NAXIS   =                    2 / 2-dimensional table                            
NAXIS1  =                  220 / width of table in bytes                        
NAXIS2  =                39812 / number of rows in table                        
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group                                 
TFIELDS =                   27 / number of columns                              
EXTNAME = 'Joined  '           / table name                                     
TTYPE1  = 'Source_Name'        / label for column 1                             
TFORM1  = '19A     '           / format for column 1                            
TCOMM1  = 'Expression: "EMUJ"+last_name'                                        
TTYPE2  = 'RA      '        

python nway-write-header.py gkvInputCatv02.fits GAMA23 82.7
python nway-write-header.py G23-ASKAP-EMUES-master-cat.fits EMU 103

python nway.py G23-ASKAP-EMUES-master-cat.fits :E_RA :E_DEC gkvInputCatv02.fits 0.1 --out=example1.fits --radius 15 --prior-completeness 0.9

In [7]:
from astropy.io import fits
import numpy as np

def calculate_survey_area(fits_file):
    # Open the FITS file and extract the header
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data
        
        # Ensure data is at least 2D (handle multi-dimensional FITS files)
        if data.ndim > 2:
            data = data[0]  # Take the first frame if it's a multi-frame FITS
        
        # Get image dimensions
        height, width = data.shape[-2:]  # Ensure correct shape indexing
        total_pixels = width * height
        
        # Extract pixel scale from header (if available)
        if 'PIXSCALE' in header:
            pixel_scale = header['PIXSCALE']  # in arcsec/pixel
        elif 'CD1_1' in header and 'CD2_2' in header:
            pixel_scale = np.abs(header['CD1_1']) * 3600  # Convert from degrees to arcseconds
        elif 'CDELT1' in header and 'CDELT2' in header:
            pixel_scale = np.abs(header['CDELT1']) * 3600  # Convert from degrees to arcseconds
        else:
            raise ValueError("Pixel scale not found in FITS header. Check metadata or specify manually.")
        
        # Compute the survey area in square arcseconds
        area_arcsec2 = total_pixels * (pixel_scale ** 2)
        
        # Convert to square arcminutes and square degrees
        area_arcmin2 = area_arcsec2 / 3600
        area_deg2 = area_arcsec2 / (3600 ** 2)
        
        return {
            'pixels': total_pixels,
            'pixel_scale (arcsec/pixel)': pixel_scale,
            'area (arcsec^2)': area_arcsec2,
            'area (arcmin^2)': area_arcmin2,
            'area (deg^2)': area_deg2
        }

# Example usage:
fits_file = "/home/polaris/Laboratório Astrofísica/data/G23-ASKAP-EMUES-data/G23-EMUES-mosaic.fits"
result = calculate_survey_area(fits_file)
print(result)


{'pixels': 214163136, 'pixel_scale (arcsec/pixel)': np.float64(2.49999999999984), 'area (arcsec^2)': np.float64(1338519599.9998288), 'area (arcmin^2)': np.float64(371810.99999995244), 'area (deg^2)': np.float64(103.28083333332012)}
